# Carga de Datos

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import warnings
# Importamos las herramientas de limpieza del Task 2 
sys.path.append('../task2') 
from data_loading import limpiar_datos
from feature_selection import seleccionar_features
from data_scaling import escalar_datos
from data_splitting import dividir_datos

warnings.filterwarnings('ignore')

# Cargar y preparar los mismos datos que en Task 3
df_clean = limpiar_datos('../task2/dataset_phishing.csv')
top_2_features, X_all, y = seleccionar_features(df_clean, n_features=2)

X_2d = df_clean[top_2_features]
X_2d_scaled, scaler_2d = escalar_datos(X_2d, y, method='standard')
X_train, X_test, y_train, y_test = dividir_datos(X_2d_scaled, y)

# Convertir a numpy para evitar warnings de sklearn
X_train_np = X_train.values
y_train_np = y_train.values
X_test_np = X_test.values
y_test_np = y_test.values

print("Datos listos para Task 4.")

✅ Datos divididos: 9144 train (80%), 2286 test (20%)
Datos listos para Task 4.


# Implementación con Scikit-Learn

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# --- 1. Regresión Logística (Sklearn) ---
sk_log_model = LogisticRegression(random_state=42)
sk_log_model.fit(X_train_np, y_train_np)
y_pred_sk_log = sk_log_model.predict(X_test_np)

# --- 2. K-Nearest Neighbors (Sklearn) ---
# Usamos k=3
sk_knn_model = KNeighborsClassifier(n_neighbors=3)
sk_knn_model.fit(X_train_np, y_train_np)
y_pred_sk_knn = sk_knn_model.predict(X_test_np)

print("Modelos de Scikit-Learn entrenados exitosamente.")

Modelos de Scikit-Learn entrenados exitosamente.


# Tabla Comparativa

In [8]:
# Función auxiliar para calcular métricas
def get_metrics(y_true, y_pred, model_name):
    return {
        'Modelo': model_name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred)
    }

metrics_list = []


# Resultados Manuales (Extraídos de Task 3) ---

# 1. Regresión Logística Manual
metrics_list.append({
    'Modelo': 'Manual - Regresión Logística',
    'Accuracy': 0.8661,   # Valor de tu output celda 15
    'Precision': 0.8727,  # Calculado: 980 / 1123
    'Recall': 0.8574      # Calculado: 980 / 1143
})

# 2. KNN Manual (k=3)
metrics_list.append({
    'Modelo': 'Manual - KNN (k=3)',
    'Accuracy': 0.5009,   # Valor de tu output celda 23
    'Precision': 1.0,     # Calculado: 2 / 2 (Pocos positivos, pero todos correctos)
    'Recall': 0.0017      # Calculado: 2 / 1143 (Detectó muy pocos ataques)
})

# --- Resultados Sklearn (Calculados en este notebook) ---
metrics_list.append(get_metrics(y_test_np, y_pred_sk_log, 'Sklearn - Regresión Logística'))
metrics_list.append(get_metrics(y_test_np, y_pred_sk_knn, 'Sklearn - KNN (k=3)'))

# Crear DataFrame y mostrar
comparison_df = pd.DataFrame(metrics_list)
print("\n=== Tabla Comparativa Final ===")
display(comparison_df)


=== Tabla Comparativa Final ===


,Modelo,Accuracy,Precision,Recall
0,Manual - Regresión Logística,0.866100,0.872700,0.857400
1,Manual - KNN (k=3),0.500900,1.000000,0.001700
2,Sklearn - Regresión Logística,0.866142,0.872663,0.857393
3,Sklearn - KNN (k=3),0.500875,1.000000,0.001750


## 2. Análisis de Cierre

### a. ¿Cuál implementación fue mejor y por qué cree que sucedió?

Se observó que la implementación manual de Regresión Logística obtuvo una exactitud de 86.61%, prácticamente idéntica a la de Scikit-Learn. Esto valida la corrección de las fórmulas matemáticas aplicadas. Sin embargo, se prefiere el uso de Scikit-Learn en producción por su eficiencia computacional y facilidad de implementación y estas diferencias clave:

1.  **Optimizaciones:** Sklearn no utiliza un descenso de gradiente simple. Implementa solucionadores avanzados como `lbfgs` o `liblinear` que ajustan el *learning rate* dinámicamente, logrando converger más rápido.

2.  **Estructuras de Datos:** Para KNN, nuestra implementación manual calcula la distancia contra todos los puntos. Sklearn utiliza estructuras de árbol KD-Trees o Ball-Trees que reducen la complejidad de búsqueda  haciendo el proceso mucho más veloz.

### b. Phishing: Falsos Positivos vs. Falsos Negativos

**Conclusión:** El error más costoso es el **Falso Negativo** por lo tanto, la métrica que deberíamos priorizar es el **Recall (Sensibilidad)**. Buscamos maximizar la detección de amenazas reales, incluso si eso implica tolerar una tasa ligeramente mayor de falsas alarmas.